In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!unzip "/content/drive/My Drive/Data/fc33077e-6-dataset.zip"

In [0]:
#!pip install catboost
#!pip install -U yellowbrick

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from yellowbrick.model_selection import FeatureImportances
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
%%time
train = pd.read_csv('/content/dataset/train.csv')

CPU times: user 35.9 ms, sys: 17.9 ms, total: 53.8 ms
Wall time: 55.4 ms


In [0]:
X = train.drop(columns = ['air_pollution_index'])
y = train['air_pollution_index']

In [7]:
X.shape

(33750, 13)

In [0]:
def prepare_df(train):
    org_keys = train.columns.tolist()
    
    train['is_holiday'] = np.where(train['is_holiday'] == 'None', False, True)
    train['date_time'] = pd.to_datetime(train['date_time'])
    
    train.set_index('date_time', inplace = True)
    
    days = ['01d', '03d', '07d', '14d', '30d']
    
    sum_attributes = ['is_holiday', 'rain_p_h', 'snow_p_h']

    for val in sum_attributes:
        for day in days:
            train['num_'+val+'_'+day] = train.rolling(day)[val].sum()
            
    weather_types = train['weather_type'].unique().tolist()
    for weather in weather_types:
        train[weather] = np.where(train['weather_type'] == weather, 1, 0)
    
    train.drop(columns = 'weather_type', inplace = True)
    
    avg_attributes = ['humidity', 'wind_speed', 'wind_direction', 'visibility_in_miles', 'dew_point', 'temperature', 
                     'rain_p_h', 'snow_p_h', 'clouds_all', 'traffic_volume'] + weather_types

    for val in avg_attributes:
        for day in days:
            train['avg_'+val+'_'+day] = train.rolling(day)[val].mean()
            
    for column in train.columns:
        if 'avg' in column:
            train[column+'_diff'] = train[column[4:-4]] - train[column]
            
    return train.reset_index()

In [9]:
%%time
X = prepare_df(X)

CPU times: user 597 ms, sys: 72.2 ms, total: 669 ms
Wall time: 674 ms


In [10]:
X.shape

(33750, 248)

In [0]:
X['date_time'] = X['date_time'].dt.hour

In [12]:
selector = VarianceThreshold(0.1)
selector.fit(X)

VarianceThreshold(threshold=0.1)

In [0]:
sel = X.columns[selector.get_support(indices=True)]

In [0]:
X = X[sel]

In [15]:
fsel = SelectFromModel(CatBoostRegressor(task_type="GPU"), max_features=100)
fsel.fit(X, y)

Learning rate set to 0.107721
0:	learn: 83.7228472	total: 44.9ms	remaining: 44.8s
1:	learn: 83.7140926	total: 85.5ms	remaining: 42.6s
2:	learn: 83.7002341	total: 129ms	remaining: 42.7s
3:	learn: 83.6893954	total: 177ms	remaining: 44s
4:	learn: 83.6766177	total: 218ms	remaining: 43.3s
5:	learn: 83.6674447	total: 254ms	remaining: 42.2s
6:	learn: 83.6409228	total: 291ms	remaining: 41.3s
7:	learn: 83.6325366	total: 323ms	remaining: 40.1s
8:	learn: 83.6265930	total: 353ms	remaining: 38.9s
9:	learn: 83.6135250	total: 387ms	remaining: 38.3s
10:	learn: 83.6058222	total: 415ms	remaining: 37.4s
11:	learn: 83.5917444	total: 440ms	remaining: 36.2s
12:	learn: 83.5823834	total: 464ms	remaining: 35.3s
13:	learn: 83.5674904	total: 487ms	remaining: 34.3s
14:	learn: 83.5441515	total: 510ms	remaining: 33.5s
15:	learn: 83.5369615	total: 532ms	remaining: 32.7s
16:	learn: 83.5309288	total: 549ms	remaining: 31.7s
17:	learn: 83.5118174	total: 574ms	remaining: 31.3s
18:	learn: 83.4991999	total: 598ms	remaining

SelectFromModel(estimator=<catboost.core.CatBoostRegressor object at 0x7f1d0db53588>,
                max_features=100, norm_order=1, prefit=False, threshold=None)

In [0]:
sel = X.columns[fsel.get_support(indices=True)]

In [17]:
len(sel)

60

In [0]:
X = X[sel]

In [0]:
rfe = RFE(RandomForestRegressor(n_jobs = -1), step = 5, verbose = 2)

In [20]:
rfe.fit(X, y)

Fitting estimator with 60 features.
Fitting estimator with 55 features.
Fitting estimator with 50 features.
Fitting estimator with 45 features.
Fitting estimator with 40 features.
Fitting estimator with 35 features.


RFE(estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                    criterion='mse', max_depth=None,
                                    max_features='auto', max_leaf_nodes=None,
                                    max_samples=None, min_impurity_decrease=0.0,
                                    min_impurity_split=None, min_samples_leaf=1,
                                    min_samples_split=2,
                                    min_weight_fraction_leaf=0.0,
                                    n_estimators=100, n_jobs=-1,
                                    oob_score=False, random_state=None,
                                    verbose=0, warm_start=False),
    n_features_to_select=None, step=5, verbose=2)

In [0]:
sel = X.columns[rfe.get_support(indices=True)]

In [0]:
X = X[sel]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [0]:
estimator = CatBoostRegressor(task_type="GPU",
                              devices='0:1')

In [25]:
estimator.fit(X_train, y_train)

Learning rate set to 0.105541
0:	learn: 83.6095838	total: 24ms	remaining: 23.9s
1:	learn: 83.5990053	total: 46.4ms	remaining: 23.1s
2:	learn: 83.5777149	total: 69.7ms	remaining: 23.2s
3:	learn: 83.5593537	total: 92.6ms	remaining: 23.1s
4:	learn: 83.5407741	total: 116ms	remaining: 23s
5:	learn: 83.5220421	total: 142ms	remaining: 23.5s
6:	learn: 83.5033943	total: 165ms	remaining: 23.4s
7:	learn: 83.4821364	total: 190ms	remaining: 23.6s
8:	learn: 83.4659756	total: 213ms	remaining: 23.4s
9:	learn: 83.4439822	total: 237ms	remaining: 23.5s
10:	learn: 83.4201614	total: 261ms	remaining: 23.4s
11:	learn: 83.4088326	total: 279ms	remaining: 22.9s
12:	learn: 83.3891929	total: 295ms	remaining: 22.4s
13:	learn: 83.3734817	total: 311ms	remaining: 21.9s
14:	learn: 83.3611324	total: 327ms	remaining: 21.5s
15:	learn: 83.3515145	total: 345ms	remaining: 21.2s
16:	learn: 83.3339725	total: 363ms	remaining: 21s
17:	learn: 83.3185891	total: 380ms	remaining: 20.7s
18:	learn: 83.3071169	total: 398ms	remaining: 

In [26]:
estimator.score(X_test, y_test)

-0.026802693359631036

In [0]:
test = pd.read_csv('/content/dataset/train.csv')

In [28]:
%%time
X_sub = prepare_df(test)

CPU times: user 1.64 s, sys: 22.9 ms, total: 1.66 s
Wall time: 894 ms


In [0]:
X_sub['date_time'] = X_sub['date_time'].dt.hour

In [0]:
X_sub = X_sub[sel]

In [0]:
test['air_pollution_index'] = estimator.predict(X_sub)

In [0]:
test = test.reset_index()

In [0]:
test.to_csv('sub1.csv', columns = ['date_time', 'air_pollution_index'], index = False)